In [ ]:
import cv2
from ultralytics import YOLO
import pyttsx3

# 초기화
model = YOLO("/Users/goremi/Downloads/[강의교안][실습]+02.+동영상+파일+다루기,+마우스+이벤트+예제+구현 (1)/fire.pt")  # fall.pt 파일이 동일 폴더에 있어야 함

engine = pyttsx3.init()
cap = cv2.VideoCapture(0)
if not cap.isOpened():

    raise RuntimeError("카메라를 열 수 없습니다")

while True:
    ret, frame = cap.read()
    if not ret: break

    # 추론
    results = model(frame, conf=0.3,verbose=False)

    # 화재 감지 플래그
    fire_detected = False

    # 결과 처리
    for result in results:
        for box in result.boxes:


            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            confidence = box.conf.item()
            # class_id = int(box.cls.item())
            # label = model.names[class_id]
            label = model.names[int(box.cls.item())]
            print(label)
            # 화재 감지 시 시각화
            if label == 'Fire':
                color = (0, 0, 255)  # 빨간색 (BGR)
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f'{label} {confidence:.2f}',
                            (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.7, color, 2)
                fire_detected = True  # 플래그 업데이트

    # 화재가 감지된 경우에만 음성 출력 (중복 방지)
    if fire_detected:
        engine.say("화재가 감지되었습니다. 즉시 대피하세요.")
        engine.runAndWait()

    cv2.imshow('Fire Detection', frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

